In [ ]:
%bazel "@PythonKit//:PythonKit"
%bazel "@SwiftCrypto//:Crypto"
%bazel "//mujoco:jupyter"

Installing packages:
|-@PythonKit//:PythonKit
|-@SwiftCrypto//:Crypto
|-//mujoco:jupyter
Working in: /home/liu/Dropbox/workspace/sim2real/.ibzlnb/eSsdO


: 

In [2]:
import PythonKit

In [3]:
%include "EnableJupyterDisplay.swift"

In [4]:
import MuJoCo
import MuJoCoJupyterExtensions

In [5]:
let simulate = Simulate(width: 960, height: 540, title: "simulate")
var model = try MjModel(fromXMLPath: "/home/liu/workspace/s4nnc/gym/assets/ant.xml")
var data = model.makeData()
simulate.use(model: model, data: &data)

In [6]:
let httpRenderServer = HTTPRenderServer(simulate, maxWidth: 960, maxHeight: 540, canResize: false)
let channel = try httpRenderServer.bind(host: "0.0.0.0", port: 12345).wait()

In [7]:
simulate.makeContext(hidden: true)
let cpugenesis = GLContext.time
let simgenesis = data.time
var cpusync = cpugenesis
var simsync = simgenesis

In [ ]:
JupyterDisplay.display(html: httpRenderServer.html)
JupyterDisplay.flush()
while true {
    model.step(data: &data)
    simsync = data.time
    while simsync - simgenesis >= cpusync - cpugenesis {
        simulate.yield()
        cpusync = GLContext.time
    }
}